In [1]:
%run data.py


In [2]:
import folium
import json
import requests

with open('data/borough.geojson') as f:
  borou = json.load(f)
san_map = folium.Map(
    location=[40.693943, -73.985880],
    tiles='Stamen Toner',
    zoom_start=10,
    min_zoom = 10,
    control_scale = True
)
#['#fee6ce','#fdae6b','#e6550d']
folium.GeoJson(
    borou,
    style_function=lambda feature: {
        'fillColor': '#ffff00',
        'color': 'blue',
        'weight': 2,
#         'dashArray': '5, 5'
    }
).add_to(san_map)
san_map

ImportError: No module named folium

In [ ]:

bdata = fetchData(nyc_boro_url)
bdata.loc[0,'BOROUGH_GROUP'] = 'Bronx' 
bdata = bdata.drop(labels=5)
bdata.to_csv('data/bdata.csv',index=False,header=True)


bdata

In [ ]:
import json
import requests

with open('data/borough.geojson') as f:
  borou = json.load(f)

nyc_map = folium.Map(
    location=[40.693943, -73.985880],
    zoom_start=10,
#     tiles='Stamen Toner',
    min_zoom = 9,
    control_scale = True
)


folium.Choropleth(
    geo_data=borou,
    data=bdata,
    name='choropleth',
    columns=['BOROUGH_GROUP', 'COVID_CASE_RATE'],
    key_on='properties.boro_name',
    fill_color='YlOrRd',
    fill_opacity=0.9,
    line_opacity=0.2,
    legend_name=' Rate per 100,000 people',

).add_to(nyc_map)


nyc_map

In [ ]:

nyc_map2 = folium.Map(
    location=[40.693943, -73.985880],
    zoom_start=10,
    tiles='cartodbpositron',
    min_zoom = 9,
    control_scale = True
)
tooltip = 'Click me!'



folium.Choropleth(
    geo_data=borou,
    data=bdata,
    name='choropleth',
  
    columns=['BOROUGH_GROUP', 'COVID_CASE_COUNT'],
    key_on='properties.boro_name',
    fill_color='YlOrRd',
    fill_opacity=0.7,
    line_opacity=0.5,
    legend_name='COVID_CASE_COUNT',

).add_to(nyc_map2)


nyc_map2

In [ ]:
# run this to create a new zdata
zdata = fetchData(nyc_zcta_url)

zdata = zdata.drop(labels=0)
zdata["MODZCTA"] = zdata["MODZCTA"].apply(lambda x : int(x))
zdata["MODZCTA"] = zdata["MODZCTA"].astype(str)

zdata.to_csv('data/zdata.csv',index=False,header=True)

zdata


In [ ]:
from bokeh.plotting import figure, output_file, show
from bokeh.tile_providers import CARTODBPOSITRON, get_provider
from bokeh.io import  output_notebook

output_notebook() 

tile_provider = get_provider(CARTODBPOSITRON)

# range bounds supplied in web mercator coordinates
px = figure(x_range=(-2000000, 6000000), y_range=(-1000000, 7000000),
           x_axis_type="mercator", y_axis_type="mercator")
px.add_tile(tile_provider)

show(px)

In [ ]:
%run data.py
# updata
# run this to create a new zdata
zdata = fetchData(nyc_zcta_url)

zdata = zdata.drop(labels=0)
zdata["MODZCTA"] = zdata["MODZCTA"].apply(lambda x : int(x))
zdata["MODZCTA"] = zdata["MODZCTA"].astype(str)

zdata.to_csv('data/zdata.csv',index=False,header=True)

zdata

In [ ]:

value  = zdata.loc[zdata['MODZCTA'] == "11371"]
a = value["Total"]
value

# for row in zdata.itertuples():
# #     print(row[1], row[3])
    

In [ ]:
import json
with open('data/nyctopo.json', 'r') as f:
    jsondata = json.load(f)
    
for item in jsondata["objects"]["zipcode"]["geometries"]:
    id = item["properties"]['zipcode']
    value  = zdata.loc[zdata['MODZCTA'] == str(id)]
    if (value.empty):
       continue    
#     print(value.iloc[0]["Total"])
    item["properties"]["test_num"] = int(value.iloc[0]["Total"])
    item["properties"]["positive_num"] = int(value.iloc[0]["Positive"])
    item["properties"]["zcta_prec"] = float(value.iloc[0]["zcta_cum.perc_pos"])
    print(item["properties"])


# with open('data/nyctop.json', 'w+') as f:
#     json.dump(jsondata, f)
    

In [ ]:
with open('data/nyctop.json', 'w+') as f:
    json.dump(jsondata, f)
    

In [106]:
import folium

with open('data/zip.geojson') as f:
  polygon = json.load(f)

zcta_map = folium.Map(
    location=[40.693943, -73.985880],
    zoom_start=10,
    tiles='cartodbpositron',
    min_zoom = 9,
    control_scale = True
)


folium.Choropleth(
    geo_data=polygon,
    data=zdata,
    name='choropleth',
    columns=['MODZCTA', 'Positive'],
    key_on='properties.postalCode',
    fill_color='YlOrRd',
    fill_opacity=0.7,
    line_opacity=0.5,
    legend_name='Positive Cases by ZIP Code',
    nan_fill_color='white',

).add_to(zcta_map)

zcta_map.save('map.html')